Confirm that all links in the people database go to working sites. 

In [1]:
import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta

import os
import sys
import csv
from tqdm import tqdm

import tweepy

consumer_key = os.environ.get('TWITTER_API_KEY')
consumer_secret = os.environ.get('TWITTER_API_KEY_SECRET')
access_key = os.environ.get('TWITTER_ACCESS_TOKEN')
access_secret = os.environ.get('TWITTER_ACCESS_TOKEN_SECRET')

In [2]:
people = pd.read_csv('../data/people.csv')
candidates = pd.read_csv('../data/candidates.csv')
candidate_statuses = pd.read_csv('../data/candidate_statuses.csv')

people_candidates = pd.merge(people, candidates, how='inner', on='person_id')
people_candidates_status = pd.merge(people_candidates, candidate_statuses, how='inner', on='candidate_status')
people_candidates_active = people_candidates_status[people_candidates_status['count_as_candidate']].copy()

## Twitter

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [4]:
people_candidates_active['twitter_handle_lower'] = (
    people_candidates_active['twitter_link']
    .str.replace('https://twitter.com/', '')
    .str.lower()
)

list_of_handles = (
    people_candidates_active.loc[people_candidates_active['twitter_handle_lower'].notnull(), 'twitter_handle_lower']
    .tolist()
)

In [5]:
batch_size = 40
num_batches = np.ceil(len(list_of_handles) / batch_size)
batches = np.array_split(list_of_handles, num_batches)

In [6]:
user_df_list = []

for batch in tqdm(batches):
    lookup_response = api.lookup_users(screen_names=list(batch))
    
    temp_df = pd.DataFrame()

    for u in lookup_response:
        temp_df.loc[u.id, 'screen_name'] = u.screen_name
        temp_df.loc[u.id, 'created_at'] = u.created_at
        temp_df.loc[u.id, 'followers_count'] = u.followers_count
        temp_df.loc[u.id, 'statuses_count'] = u.statuses_count
        temp_df.loc[u.id, 'verified'] = u.verified
        temp_df.loc[u.id, 'protected'] = u.protected


    user_df_list += [temp_df]
    
    time.sleep(5)

100%|██████████| 5/5 [00:29<00:00,  5.85s/it]


In [7]:
user_df = pd.concat(user_df_list)

In [8]:
user_df['screen_name_lower'] = user_df['screen_name'].str.lower()
user_df

,screen_name,created_at,followers_count,statuses_count,verified,protected,screen_name_lower
1946942610,LaylaBonnot,2013-10-08 13:53:51,937.0,2911.0,False,False,laylabonnot
182387417,zach_rybarczyk,2010-08-24 13:47:59,122.0,418.0,False,True,zach_rybarczyk
806545555094704128,MillerANC1A05,2016-12-07 17:07:00,713.0,3266.0,False,False,milleranc1a05
330233286,KentBoeseDC,2011-07-06 09:36:50,2442.0,8447.0,False,False,kentboesedc
874536666,mwraydc,2012-10-11 23:46:58,434.0,1449.0,False,False,mwraydc
...,...,...,...,...,...,...,...
2337449737,PranavNandaDC,2014-02-11 00:12:21,671.0,4370.0,False,False,pranavnandadc
229348030,thisisbossi,2010-12-22 05:10:30,1032.0,48303.0,False,False,thisisbossi
1304661831953395712,Yannikfor2A01,2020-09-12 06:03:24,156.0,192.0,False,False,yannikfor2a01
44801505,EdwardRyder,2009-06-05 02:41:23,390.0,120.0,False,False,edwardryder


In [9]:
working_screen_name_lower = [s.lower() for s in user_df['screen_name'].tolist()]

In [10]:
missing_users = [f for f in list_of_handles if f not in working_screen_name_lower]

In [11]:
missing_users

[]

In [12]:
len(missing_users)

0

In [13]:
len(list_of_handles)

174

### Superlatives

In [14]:
user_df.sort_values(by='followers_count')

,screen_name,created_at,followers_count,statuses_count,verified,protected,screen_name_lower
1305169316464259072,AncMeg,2020-09-13 15:40:28,0.0,3.0,False,False,ancmeg
1154769474261856257,Christo34514033,2019-07-26 15:04:32,0.0,5.0,False,False,christo34514033
1252919319111053313,SteveHoltzman3,2020-04-22 11:18:03,1.0,1.0,False,False,steveholtzman3
1288727768893710337,EKlossou,2020-07-30 06:47:10,4.0,28.0,False,False,eklossou
227554859,PierreCHines,2010-12-17 04:06:55,4.0,4.0,False,False,pierrechines
...,...,...,...,...,...,...,...
803360187125993472,riotpedestrian,2016-11-28 22:09:29,3956.0,15972.0,False,False,riotpedestrian
33211889,keya_chatterjee,2009-04-19 14:33:28,4025.0,8529.0,False,False,keya_chatterjee
2196941238,salimadofo,2013-11-16 02:07:57,4081.0,43798.0,True,False,salimadofo
40059261,sabelharris,2009-05-14 18:53:42,5229.0,8762.0,False,False,sabelharris


In [15]:
user_df.sort_values(by='statuses_count')

,screen_name,created_at,followers_count,statuses_count,verified,protected,screen_name_lower
47946516,BTurmail,2009-06-17 12:39:56,13.0,0.0,False,False,bturmail
1285376801376698368,LatoyaM01766637,2020-07-21 00:51:35,6.0,0.0,False,False,latoyam01766637
1252919319111053313,SteveHoltzman3,2020-04-22 11:18:03,1.0,1.0,False,False,steveholtzman3
1305169316464259072,AncMeg,2020-09-13 15:40:28,0.0,3.0,False,False,ancmeg
1027034717139419138,Anc7B07,2018-08-08 03:32:12,118.0,3.0,False,False,anc7b07
...,...,...,...,...,...,...,...
170780048,GeorgetownMet,2010-07-25 19:06:48,2082.0,30663.0,False,False,georgetownmet
716998860548214785,Ryanfor3F05,2016-04-04 14:40:24,870.0,31168.0,False,False,ryanfor3f05
2196941238,salimadofo,2013-11-16 02:07:57,4081.0,43798.0,True,False,salimadofo
229348030,thisisbossi,2010-12-22 05:10:30,1032.0,48303.0,False,False,thisisbossi


In [16]:
user_df[user_df['verified']]

,screen_name,created_at,followers_count,statuses_count,verified,protected,screen_name_lower
1921416134,annalandre,2013-09-30 21:53:15,3066.0,2366.0,True,False,annalandre
2890872052,tmholcomb,2014-11-24 14:43:44,1027.0,1864.0,True,False,tmholcomb
2196941238,salimadofo,2013-11-16 02:07:57,4081.0,43798.0,True,False,salimadofo
305704907,MoniDiop,2011-05-26 17:22:35,948.0,6049.0,True,False,monidiop


In [17]:
user_df[user_df['protected']]

,screen_name,created_at,followers_count,statuses_count,verified,protected,screen_name_lower
182387417,zach_rybarczyk,2010-08-24 13:47:59,122.0,418.0,False,True,zach_rybarczyk
31623503,vjsthoughts,2009-04-16 03:16:52,522.0,4899.0,False,True,vjsthoughts
1011926797787762688,LaneiceMoore,2018-06-27 10:58:44,21.0,320.0,False,True,laneicemoore
352608476,MsGorJuss,2011-08-10 21:19:56,633.0,30310.0,False,True,msgorjuss


In [18]:
user_df['followers_count'].median()

272.5

In [19]:
user_df['statuses_count'].median()

454.0

In [20]:
user_df['created_days_ago'] = (datetime.utcnow() - user_df['created_at']).dt.days
(datetime.utcnow() - timedelta(days=user_df['created_days_ago'].median())).strftime('%Y-%m-%d')

'2014-04-06'

## Websites

In [21]:
def check_status_of_links(df, link_column):
    
    bad_sites = []

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        if pd.isnull(row[link_column]):
            continue

        r = requests.get(row[link_column])

        if r.status_code != 200:
            bad_sites += [row[link_column]]

        time.sleep(1)
        
    return bad_sites

In [22]:
check_status_of_links(people_candidates_active, 'website_link')

100%|██████████| 417/417 [02:55<00:00,  2.38it/s]


['https://meghanforanc.com', 'https://www.robbdooling.com/']

In [23]:
requests.get('http://robbdooling.com')

<Response [406]>

In [24]:
requests.get('https://meghanforanc.com')

<Response [406]>

## Facebook

In [25]:
check_status_of_links(people_candidates_active, 'facebook_link')

100%|██████████| 417/417 [02:00<00:00,  3.45it/s]


[]